# Clonamos el repositorio con los modelos y herramientas

In [1]:
!git clone https://github.com/dannasalazar11/Msc_thesis.git

Cloning into 'Msc_thesis'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 97 (delta 42), reused 53 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (97/97), 37.92 KiB | 1.90 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [2]:
import sys
sys.path.append('/kaggle/working/Msc_thesis')

from gmrrnet_adhd.utils import get_segmented_data, train_L24O_cv
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

2025-04-29 15:12:06.650926: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745939526.896076      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745939526.964774      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Importar base de datos segmentada (Segmentos de 4 seg con translape del 50%, es decir, de 2 seg)

In [3]:
X, y, sbjs = get_segmented_data()
X.shape, y.shape, len(sbjs)

((8213, 19, 512), (8213, 2), 8213)

# Importamos el modelo y definimos los hiperparámetros

In [4]:
from tensorflow.keras.losses import CategoricalCrossentropy, MeanSquaredError
from gmrrnet_adhd.models.GMRRNet import GMRRNet

model_name = 'GMRRNet'
model_args = {
    'num_kernels': 3,
    'nb_classes': 2,
    'Chans': 19,
    'Samples': 512,
    'kernLength': 32,
    'norm_rate': 0.1,
    'alpha': 2
}


compile_args = {
    'loss':['binary_crossentropy', None], 
    'optimizer': 'adam',
    'loss_weights' : [0.9, 0.1],
    'metrics' : [['binary_accuracy'], [None]]
}

model = GMRRNet(**model_args)

model.summary()

I0000 00:00:1745939544.897542      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1745939544.898123      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 19, 512)        │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cast (Cast)               │ (None, 19, 512)        │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape (Reshape)         │ (None, 512, 19)        │              0 │ cast[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization       │ (None, 512, 19)        │             38 │ reshape[0][0]          │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder       │ (None, 512, 19)        │          6,528 │ layer_normalization[0… │
│ (TransformerEncoder)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_1     │ (None, 512, 19)        │             38 │ transformer_encoder[0… │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape_1 (Reshape)       │ (None, 19, 512, 1)     │              0 │ layer_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gaussian_layer_1          │ (None, 19, 19, 1)      │              0 │ reshape_1[0][0]        │
│ (GaussianKernelLayer)     │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gaussian_layer_2          │ (None, 19, 19, 1)      │              0 │ reshape_1[0][0]        │
│ (GaussianKernelLayer)     │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gaussian_layer_3          │ (None, 19, 19, 1)      │              0 │ reshape_1[0][0]        │
│ (GaussianKernelLayer)     │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 19, 19, 5)      │             50 │ gaussian_layer_1[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 19, 19, 5)      │             50 │ gaussian_layer_2[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 19, 19, 5)      │             50 │ gaussian_layer_3[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 19, 19, 15)     │              0 │ conv2d[0][0],          │
│ (Concatenate)             │                        │                │ conv2d_1[0][0],        │
│                           │                        │                │ conv2d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv2D_2 (Conv2D)         │ (None, 19, 19, 3)      │            408 │ concatenate_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization  

 Total params: 9,438 (36.87 KB)

 Trainable params: 9,426 (36.82 KB)

 Non-trainable params: 12 (48.00 B)

# Resultados - Leave 24 Subjects Out +  CV (cada fold se corre 5 veces)

In [5]:
import os

import pickle

with open("/kaggle/input/ieee-tdah-control-database/folds.pkl", "rb") as f:
    folds = pickle.load(f)

In [6]:
results = train_L24O_cv(GMRRNet, X, y, sbjs, model_args, compile_args, folds,model_name)

Fold 1/5. Test subjects: ['v28p', 'v274', 'v1p', 'v231', 'v22p', 'v29p', 'v206', 'v238', 'v31p', 'v35p', 'v177', 'v200', 'v112', 'v113', 'v48p', 'v140', 'v131', 'v125', 'v55p', 'v143', 'v43p', 'v305', 'v134', 'v114']


I0000 00:00:1745939556.974749      73 service.cc:148] XLA service 0x7cba2c001640 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745939556.975632      73 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1745939556.975651      73 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1745939558.140887      73 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1745939566.526171      73 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Fold metrics: {'accuracy': 0.6358956694602966, 'recall': 0.6352679852583564, 'precision': 0.6354172967006381, 'kappa': 0.27066883769427796, 'auc': 0.6352679852583564}
Fold 2/5. Test subjects: ['v18p', 'v39p', 'v234', 'v32p', 'v190', 'v6p', 'v254', 'v204', 'v24p', 'v183', 'v246', 'v219', 'v298', 'v41p', 'v47p', 'v308', 'v52p', 'v300', 'v59p', 'v299', 'v302', 'v51p', 'v109', 'v127']
Fold metrics: {'accuracy': 0.8009592294692993, 'recall': 0.793564644310913, 'precision': 0.8010947895848499, 'kappa': 0.5926590118353205, 'auc': 0.7935646443109129}
Fold 3/5. Test subjects: ['v215', 'v3p', 'v209', 'v37p', 'v213', 'v15p', 'v284', 'v181', 'v19p', 'v34p', 'v263', 'v244', 'v138', 'v121', 'v46p', 'v54p', 'v120', 'v310', 'v147', 'v50p', 'v56p', 'v107', 'v297', 'v108']
Fold metrics: {'accuracy': 0.8848981857299805, 'recall': 0.8612904732357906, 'precision': 0.9037724791109614, 'kappa': 0.750174898074251, 'auc': 0.8612904732357906}
Fold 4/5. Test subjects: ['v227', 'v8p', 'v236', 'v14p', 'v196', 'v27

In [7]:
import numpy as np

accs = []
for result in results:
    accs.append(result['accuracy'])

np.mean(accs)

0.7825037717819214

In [8]:
import pickle

with open(f'results_L24SO_{model_name}.pkl', 'wb') as f:
    pickle.dump(results, f)